In [ ]:
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("talk")
import platform

%load_ext autoreload
%autoreload 2

In [ ]:
from kalman_steps import *
from helpers import *

In [ ]:
TIME_INTERVAL_TO_USE = "1s"
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# Convert the time interval string to a pandas Timedelta object
time_interval = pd.Timedelta(TIME_INTERVAL_TO_USE)
# Extract the total seconds from the Timedelta object and convert to float
time_interval_in_seconds = time_interval.total_seconds()
print(f"Time interval in seconds: {time_interval_in_seconds}")

In [ ]:
file_identifiers = load_identifiers();
pick_ident = "kyzikos_run"
file_name = file_identifiers[pick_ident]["file_name"]
file_ident = file_identifiers[pick_ident]
additional_run_info = {}

In [ ]:
gpx_obj, segment = step_01_load_data(file_name)

In [ ]:
print_known_info(file_ident, ["total_elevation"])
segment.get_uphill_downhill()

In [ ]:
coords = pd.DataFrame([{'lat': p.latitude, 
                        'lon': p.longitude, 
                        'ele': p.elevation,
                        'time': p.time} for p in segment.points])
coords.set_index('time', drop=True, inplace=True)
coords.head()

In [ ]:
for point in segment.points:
    point.elevation = None

In [ ]:
import srtm
elevation_data = srtm.get_data()
elevation_data.add_elevations(gpx_obj, smooth=True)

In [ ]:
coords['new_ele'] = [p.elevation for p in gpx_obj.tracks[0].segments[0].points]

In [ ]:
segment.get_uphill_downhill()

In [ ]:
coords.head()

In [ ]:
_, distance_vincenty = calc_speed_distance_vincent(segment, verbose=False)
print(f"in {len(distance_vincenty)} points max distance between 2 points is {np.max(distance_vincenty):4.2f} whereas min is {np.min(distance_vincenty[distance_vincenty>0]):4.2f}")

In [ ]:
total_elevation_change = sum(abs(coords['new_ele'][1:]-coords['new_ele'][0:-1]))/2
print(f"total_elevation_change={total_elevation_change:4.2f} meters")

In [ ]:
elev_acquired=coords['new_ele'][1:]-coords['new_ele'][0:-1]
elev_acquired = elev_acquired.reset_index(drop=True)
elev_values = elev_acquired.tolist()

dist_acquired = distance_vincenty[1:]
angles = np.arctan2(elev_values, dist_acquired)
dist_acc = np.cumsum(dist_acquired)
elev_acc = np.cumsum(elev_values)

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(18,12))
axs[0,0].plot(elev_values)
axs[0,0].set_title("elev_values")
xticks = np.arange(0,len(elev_values),100)
xlabels = [i for i in xticks]
axs[0,1].plot(elev_acc)
axs[0,1].set_title("elev_acc")
axs[1,0].plot(dist_acc)
axs[1,0].set_title("dist_acc")
axs[1,1].plot(angles)
axs[1,1].set_title("angles")
for i in range(2):
    for j in range(2):
        axs[i,j].set_xticks(xticks)
        axs[i,j].set_xticklabels(xlabels, rotation=45)
plt.tight_layout()

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(12,10))
fr=0
to=100
title_add = f"{fr}:{to}"
tick_inc = 20
axs[0,0].plot(elev_values[fr:to])
axs[0,0].set_title(f"elev_values:{title_add}")
xticks = np.arange(0,len(elev_values[fr:to]),tick_inc)
xlabels = [i for i in xticks]
axs[0,1].plot(elev_acc[fr:to])
axs[0,1].set_title(f"elev_acc:{title_add}")
axs[1,0].plot(dist_acc[fr:to])
axs[1,0].set_title(f"dist_acc:{title_add}")
axs[1,1].plot(angles[fr:to])
axs[1,1].set_title(f"angles:{title_add}")
for i in range(2):
    for j in range(2):
        axs[i,j].set_xlim(fr,to)
        axs[i,j].set_xticks(xticks)
        axs[i,j].set_xticklabels(xlabels, rotation=45)
plt.tight_layout()
#plt.plot()

fig = plt.figure(figsize=(10,6))
plt.plot(elev_acc[0:500])
x = [20, 62, 151, 264, 275, 305, 327, 446, 461]
plt.scatter(x,np.zeros(len(x)))

fig, axs = plt.subplots(4)
pick_until = 500
axs[0].plot(angles[0:pick_until])
block_sizes = [5, 10, 20]
added_angles_0 = np.convolve(angles[0:pick_until],np.ones(block_sizes[0],dtype=int),'valid')
axs[1].plot(added_angles_0)
added_angles_1 = np.convolve(angles[0:pick_until],np.ones(block_sizes[1],dtype=int),'valid')
axs[2].plot(added_angles_1)
added_angles_2 = np.convolve(angles[0:pick_until],np.ones(block_sizes[2],dtype=int),'valid')
axs[3].plot(added_angles_2)

tresh=10
changes_row2 = find_direction_change_indices(added_angles_0, tresh=tresh)
axs[1].scatter(np.squeeze(changes_row2), np.zeros(len(changes_row2)))
changes_row3 = find_direction_change_indices(added_angles_1, tresh=tresh)
axs[2].scatter(np.squeeze(changes_row3), np.zeros(len(changes_row3)))
changes_row4 = find_direction_change_indices(added_angles_2, tresh=tresh)
axs[3].scatter(np.squeeze(changes_row4), np.zeros(len(changes_row4)))


plt.tight_layout()


In [ ]:
from helpers import find_direction_change_indices
fig, axs = plt.subplots(3,3,figsize=(12,10))
pick_until = 500
change_points = {}
for i in range(3):
    for j in range(3):
        block_size = 10*(i+1)
        tresh = 5*(j+1)
        title_str = f"block_size:{block_size},tresh:{tresh}"
        added_angles = np.convolve(angles[0:pick_until],np.ones(block_size,dtype=int),'valid')
        cpts = find_direction_change_indices(added_angles, tresh=tresh)
        change_points[title_str] = cpts
        axs[i,j].plot(added_angles)
        axs[i,j].scatter(np.squeeze(cpts), np.zeros(len(cpts)), s=20, c='red', marker='o')
        axs[i,j].set_title(title_str)
        #axs[i,j].set_xticks(xticks)
        #axs[i,j].set_xticklabels(xlabels, rotation=45)
plt.tight_layout()

In [ ]:
print(np.column_stack((dist_acquired[0:50], elev_values[0:50],angles[0:50])))

In [ ]:
X = angles
print(f"len of X = {len(X)}")
change_idx_X = np.squeeze(np.argwhere(X[1:]*X[:-1]<0))
treshDist = 50 #if a block is less than 25 meters long - discard it
treshElev = 2
modesDetected = []
wasDiscarded = np.zeros(len(change_idx_X))
new_change_idx = [0]
fr = 0
discardThis = False
added_mode = 'NONE'
for idAt in range(0,len(change_idx_X)):
    
    wasDiscardedPrev = idAt>1 and wasDiscarded[idAt-2]
    wasDiscardedCurrent = idAt>0 and wasDiscarded[idAt-1]
    
    i = change_idx_X[idAt]
    to = i-1
    mode = '?' 
    if (X[i]<0 and X[i+1]>0):
        mode = 'dec' 
    if (X[i]>0 and X[i+1]<0):
        mode = 'INC' 
    dist = np.sum(dist_acquired[fr:to])
    elevIncDec = np.sum(elev_values[fr:to])
    elevChange = np.sum(np.abs(elev_values[fr:to]))
    angle_of_block = (180/np.pi)*np.arctan2(elevIncDec, dist)
    discardDueToBlockLength = dist<treshDist
    discardDueToElevChange = elevChange<treshElev
    
    discardDueToPreviousDiscard = wasDiscardedCurrent and not wasDiscardedPrev
    
    if (np.abs(angle_of_block)<1):
        mode = 'FLT' 
    
    modesDetected.append(mode)
    
    discardString = ""
    if discardDueToBlockLength:
        discardString += "BL,"
    if discardDueToElevChange:
        discardString += "EC,"
    if discardDueToPreviousDiscard:
        discardString += "PD,"
        
    # now this block can be either INC dec or FLT
    # it should be needed to be discarded or not
    # first of all consecutive FLT blocks needs to be combined
        
    wasDiscarded[idAt] = discardDueToBlockLength  or discardDueToElevChange
    
    print(f"{discardString:9s}-({i}:fr({fr:3d}):to({to:3d}):m({mode}):dist({dist:8.4f}):aob({angle_of_block:8.4f}),elevChange({elevChange:5.3f}),elevIncDec({elevIncDec:5.3f})")
    if (idAt==0):
        print(f"***add-to {to} as first")
        new_change_idx.append(to)
    if idAt>0:
        #now lets see if we will add i-1 as change point
        mode_was = modesDetected[idAt-1]
        mode_is = mode
        if wasDiscarded[idAt]:
            pass
        elif (mode_was=="FLT") and (mode_is=="FLT"):
            pass
        elif (added_mode==mode_is):
            pass
        elif (mode_was!="FLT") and (mode_is=="FLT"):
            new_change_idx.append(fr)
            added_mode = "FLT"
            print(f"***add {fr} because it changed to FLAT")
        elif not wasDiscarded[idAt]:
            new_change_idx.append(fr)
            added_mode = mode_is
            print(f"***add {fr} because {mode_is} is not discarded")
    fr = i
print("new_change_idx:",new_change_idx)

In [ ]:
fr = 0
for idAt in range(1,len(new_change_idx)):  
    i = new_change_idx[idAt]
    to = i-1
    
    dist = np.sum(dist_acquired[fr:to])
    elevIncDec = np.sum(elev_values[fr:to])
    V = np.array(elev_values[fr:to])
    elevInc = np.sum(V[V>0])
    elevDec = -np.sum(V[V<0])
    angle_of_block_up = (180/np.pi)*np.arctan2(elevInc, dist)
    angle_of_block_down = (180/np.pi)*np.arctan2(elevDec, dist)
    angle_of_block = max(angle_of_block_up, angle_of_block_down)
    
    mode = 'FLT' 
    if (elevDec>treshElev and elevDec>elevInc and angle_of_block_down>=0.2):
        mode = 'dec' 
    if (elevInc>treshElev and elevInc>elevDec and angle_of_block_up>=0.2):
        mode = 'INC'     
    
    str0 = f"dist({dist:8.3f})"
    str1 = f"elevChange({max(elevInc,elevDec):6.2f})"
    str2 = f"elevInc({elevInc:6.2f})"
    str3 = f"elevDec({elevDec:6.2f})"
    print(f"{idAt:4d}:{i:4d}:fr({fr:4d}):to({to:4d}):m({mode}):{str0:12}:aob({angle_of_block:8.4f}),{str1:16},{str2:9},{str3:9}")
    fr = i